In [1]:
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl
from pprint import pprint 

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [2]:
# 8-5
class NER_tokenizer(BertJapaneseTokenizer):
       
    def encode_plus_tagged(self, text: str, entities: list, max_length: int) -> dict:
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。正解ラベルの作成を行う
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        entities = sorted(entities, key=lambda x: x['span'][0])
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # 固有表現ではないものには0のラベルを付与
            splitted.append({'text':text[position:start], 'label':0}) 
            # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
            splitted.append({'text':text[start:end], 'label':label}) 
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ] # 長さ0の文字列は除く
        # print(splitted)

        # 分割されたそれぞれの文字列をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # トークンのラベルを追加していく
        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']
            tokens_splitted = tokenizer.tokenize(text) # 形態素解析
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted) # ["word1", "word2", "word3"]
            labels.extend(labels_splitted) # [0, 0, 1, 1, 0, 0, 0]

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        encoding = tokenizer.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        ) # input_idsをencodingに変換
        # 特殊トークン[CLS]、[SEP]のラベルを0にする。
        labels = [0] + labels[:max_length-2] + [0] 
        # 特殊トークン[PAD]のラベルを0にする。
        labels = labels + [0]*( max_length - len(labels) ) 
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text: str, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word) # トークン処理後の生データを入れる
            if tokens_word[0] == '[UNK]': # トークン処理後にクレンジングをして入れる
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # print(tokens)
        # print(tokens_original)

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l]) # [[0, 2], [3, 4] ....]
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = tokenizer.convert_tokens_to_ids(tokens) 
        encoding = tokenizer.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    def convert_bert_output_to_entities(self, text: str, labels: list, spans: list) -> list:
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        entitiesの作成。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            # print(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [3]:
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

text = '昨日のみらい事務所との打ち合わせは順調だった。'
entities = [
    {'name': 'みらい事務所', 'span': [3,9], 'type_id': 1}
]

encoding = tokenizer.encode_plus_tagged(
    text, entities, max_length=20
)

print(encoding)

{'input_ids': [2, 10271, 28486, 5, 546, 10780, 2464, 13, 5, 1878, 2682, 9, 10750, 308, 10, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [5]:
%%bash 
pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

Cloning into 'ner-wikipedia-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 9), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (32/32), 663.49 KiB | 521.00 KiB/s, done.
Note: switching to 'f7ed83626d90e5a79f1af99775e4b8c6cba15295'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [4]:
class LoadWikipedia:
    def __init__(self, model_name, filename):
        """
        wikipediaのデータセット読み込み専用のクラスです。
        独自のデータセットより学習する場合は、dateloader(バッチ分割)した訓練、検証、テストデータをそれぞれ返してください
        
        """
        self.dataset = []
        self.tokenizer = NER_tokenizer.from_pretrained(model_name)
        self.type_id_dict = {
            "人名": 1,
            "法人名": 2,
            "政治的組織名": 3,
            "その他の組織名": 4,
            "地名": 5,
            "施設名": 6,
            "製品名": 7,
            "イベント名": 8
        }
        self.filename = filename 
    def _load_dataset(self):
        dataset = json.load(open(self.filename, "r"))
        for sample in dataset:
            del sample["curid"]
            sample['text'] = unicodedata.normalize('NFKC', sample['text'])
            for e in sample["entities"]:
                e['type_id'] = self.type_id_dict[e['type']]
                del e['type']
        self.dataset = dataset
    def create_dataset(self, max_length: int, batch_size: int, cuda: bool):
        self._load_dataset()
        random.shuffle(self.dataset)
        n = len(self.dataset)
        n_train = int(n*.6)
        n_val = int(n*.2)
        dataset_train = self.dataset[:n_train]
        dataset_val = self.dataset[n_train:n_train+n_val]
        dataset_test = self.dataset[n_train+n_val:]
        # bertに入力できるデータに変換する
        dataloader_train = self._create_dataloader(dataset_train, max_length, cuda)
        dataloader_val = self._create_dataloader(dataset_val, max_length, cuda)
#         dataloader_test = self._create_dataloader(dataset_test, max_length, cuda)
        # バッチ単位で分割する
        dataloader_train = DataLoader(
            dataloader_train, batch_size=batch_size, shuffle=True
        ) # (batch_size, max_length, 1)
        dataloader_val = DataLoader(
            dataloader_val, batch_size=256
        )
#         dataloader_test = DataLoader(
#             dataloader_test, batch_size=256
#         )
        
        # デバック用
#         for t in dataloader_train:
#             print(t["input_ids"].size())
#             break
        return dataloader_train, dataloader_val, dataset_test
        
        
    def _create_dataloader(self, dataset: list, max_length: int, cuda: bool):
        dataloader = []
        for data in dataset:
            text = data["text"]
            entities = data["entities"]
            # textのトークン化、正解ラベルの付与
            encoding = self.tokenizer.encode_plus_tagged(
                text, entities, max_length=max_length
            )
            encoding = {k: torch.tensor(v) for k, v in encoding.items()}
            if cuda:
                encoding = {k: v.cuda() for k, v in encoding.items()}
            dataloader.append(encoding)
        return dataloader 

In [6]:
# PyTorch Lightningのモデル
class BertForTokenClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name: str, num_labels: int, lr: float):
        """
        `BertForTokenClassification`を使った固有表現抽出タスクを実行します。
        モデルの層では、
          (文章数, 単語数, 1) 入力層
            -> (文章数, 単語数, 768) BertModelによる単語埋め込み処理
              -> (文章数, 単語数, )
        """
        super().__init__()
        self.save_hyperparameters()
        # (32, 128, 1) -> (32, 128, 768) -> (32, 128, 4)
        self.bert_tc = BertForTokenClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        print(self.bert_tc.config)
        print("~"*100)
        
    def training_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [10]:
from pprint import pprint 
class MyBertForTokenClassification_from_wikipedia:
    def __init__(self,
                 model_name="cl-tohoku/bert-base-japanese-whole-word-masking",
                 filename="ner-wikipedia-dataset/ner.json"):
        self.model_name = model_name 
        self.filename = filename 
        self.wiki = LoadWikipedia(model_name, filename)
        self.type_id_dict = self.wiki.type_id_dict
        self.model = None
        self.best_model_path = ""
        self.max_length = 0
        self.i = 0
        self.tokenizer = None
    def _trainer(self, cuda: bool):
        """
        pytorch-lightningの学習時の`Trainer`インスタンスを生成します。
        学習時にcheckpointを記録し、検証データのlossを指標とします。
        """
        checkpoint = pl.callbacks.ModelCheckpoint(
            monitor='val_loss',
            mode='min',
            save_top_k=1,
            save_weights_only=True,
            dirpath='model/'
        )

        trainer = pl.Trainer(
            max_epochs=1,
            callbacks=[checkpoint],
            gpus:1 if cuda else 0
        )
        return trainer, checkpoint
    def _eval(self, entities_pred_list, entities_list):
        """
        テストデータを使ってモデルの評価関数を作成します。
        評価条件としては、１文章当たりのentitiesが予測したentitiesの数分だけ正解とします。
        例: 
            + correct [1, 1, 0]
            + predict [1, 0]
            この場合はcorrect == predict が最初のみ正解のため,
            　　正解率は1/3(33.33%)になります。
                適合率は1/2(50.0%) -- 予測したものの中で正解だったデータ数
                再現率は1/3(33.33%) -- 正解ラベルのうち予測が正解だったデータ数
            
        """
        num_all, num_pred, num_correct = 0, 0, 0
        for entities, pred in zip(entities_list, entities_pred_list):
            get_span_id = lambda e: (e["span"][0], e["span"][1], e["type_id"])
            set_entities = set(get_span_id(e) for e in entities)
            set_entities_pred = set(get_span_id(e) for e in pred)
            
            num_all += len(entities) # 全entitiesの数
            num_pred += len(pred) # 予測数の累計
            num_correct += len(set_entities & set_entities_pred) # 予測が正しかった数の累計
            
        precision = num_correct/num_pred 
        recall = num_correct/num_all
        f_value = 2*precision*recall/(precision+recall)
        
        result = {
            'num_entities': num_all,
            'num_predictions': num_pred,
            'num_correct': num_correct,
            'precision': precision,
            'recall': recall,
            'f_value': f_value
        }
        print("テストデータの評価指標: \n")
        pprint(result)
    def _test(self, test, cuda):
        entities_list , entities_pred_list = [], []
        for i, data in enumerate(tqdm(test)):
            text = data["text"]
            entities_predict = self._predict_(text, cuda)
            entities_pred_list.append(entities_predict) # 予測
            entities_list.append(data["entities"]) # 正解
        self._eval(entities_pred_list, entities_list)
            
    def train(self, max_length=128, batch_size=32, cuda=False, lr=0.01):
        """
        BertForTokenClassificationのファインチューニング学習をします。
        wikipediaのデータセットを使用します。
        独自のデータセットで学習を行い場合には、訓練と検証データは`DataLoader`型で、
        テストデータは{"text", "entities"}をキーとするデータのリスト型で返してください。
        """
        self.max_length = max_length
        # dataloaderの取得
        train, val, test = self.wiki.create_dataset(max_length, batch_size, cuda)
        num_labels = len(self.type_id_dict)+1 # 出力層のclass数
        # モデルの読み込み
        self.model = BertForTokenClassification_pl(self.model_name, 
                                                   num_labels=num_labels,
                                                   lr=lr)
        if cuda:
            self.model = self.model.cuda()
        # 学習
        trainer, checkpoint = self._trainer(cuda)
        trainer.fit(self.model, train, val)
        # 最もlossの少ないモデルの保存
        self.best_model_path = checkpoint.best_model_path 
        self._test(test, cuda)
        print(f"best_model_path: {self.best_model_path}")
        self.model = None 
    def _load_model_(self, cuda: bool):
        if self.best_model_path != "":
            model = BertForTokenClassification_pl.load_from_checkpoint(
                self.best_model_path
            )
            self.model = model.bert_tc # bertのモデルとcheckpointの適合
            if cuda:
                self.model = self.model.cuda()
        else:
            raise NotImplementedError
            
    def _predict_(self, text: str, cuda=False):
        """
        学習時のテストデータの推論を行います。
        """
        # wikiクラスにあるtokenizerをそのまま使用する
        self.i += 1 
        if self.i == 1:
            self._load_model_(cuda)
            self.tokenizer = self.wiki.tokenizer
        encoding, spans = self.tokenizer.encode_plus_untagged(
            text, return_tensors="pt"
        )
        if cuda:
            encoding = {k: v.cuda() for k, v in encoding.items()}
        # 推論
        with torch.no_grad():
            output = self.model(**encoding)
            scores = output.logits # トークンごとの属するクラスの確率
            predict = scores[0].argmax(-1).cpu().numpy().tolist()
        # 文章から予測クラスに従って、固有表現のリストを取得する
        entities = self.tokenizer.convert_bert_output_to_entities(
            text, predict, spans
        )
        
    def predict(self, text: str, filepath="", cuda=False):
        """
        wikipediaで学習したモデルを使って、文章から固有表現の抽出を行います。
        `text`に文章を入力して実行してください。
        gpuも指定できますが、事前学習済モデルのため多くの場合で必要ありません。
        """
        self._load_model(filepath, cuda)
        self.tokenizer = self.wiki.tokenizer 
        encoding, span = self.tokenizer.encode_plus_untagged(
            text, return_tensors="pt"
        )
        if cuda:
            encoding = {k: v.cuda() for k, v in encoding.items()}
        with torch.no_grad():
            output = self.model(**encoding)
            scores = output.logits 
            predict = scores[0].argmax(-1).cpu().numpy().tolist()
        entities = self.tokenizer.convert_bert_output_to_entities(
            text, predict, spans
        )
        category = self.wiki.type_id_dict
        for entity in entities:
            entity_list = entity[0]
            name = entity_list["name"]
#             span = entity_list["span"]
            type_id = entity_list["type_id"]
            cate = category[type_id]
            print("--")
            print(f"name: {name}")
            print(f"category: {cate}")
        self.model = None 
    def _load_model(self, filepath, cuda):
        if filepath != "":
            model = BertForTokenClassification_pl.load_from_checkpoint(
                filepath
            )
            self.model = model 
            if cuda:
                self.model = self.model.cuda()
        else:
            raise NotImplementedError
        
        

In [13]:
model = MyBertForTokenClassification_from_wikipedia()

In [14]:
model.train()

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name    | Type                       | Params
-------------------------------------------------------
0 | bert_tc | BertForTokenClassification | 110 M 
-------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
440.135   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [19]:
a = torch.rand((3, 120, 4))

a.argmax(-1)

tensor([[1, 3, 2, 1, 1, 1, 0, 3, 3, 1, 0, 2, 0, 2, 1, 3, 2, 2, 2, 1, 3, 2, 3, 0,
         0, 3, 2, 2, 3, 0, 1, 0, 2, 3, 3, 3, 1, 2, 1, 2, 0, 2, 3, 2, 3, 2, 1, 0,
         2, 1, 3, 2, 1, 0, 1, 0, 1, 2, 0, 1, 3, 2, 1, 3, 1, 0, 1, 3, 1, 2, 1, 1,
         2, 0, 0, 0, 3, 0, 1, 1, 2, 1, 2, 0, 0, 0, 1, 3, 1, 2, 3, 3, 3, 0, 2, 2,
         2, 0, 3, 0, 0, 2, 0, 2, 2, 2, 3, 1, 2, 2, 2, 2, 1, 2, 3, 0, 1, 0, 2, 2],
        [1, 1, 2, 0, 2, 3, 0, 1, 0, 0, 0, 1, 1, 0, 3, 0, 2, 3, 2, 3, 1, 2, 2, 0,
         3, 0, 2, 1, 0, 2, 2, 2, 1, 2, 3, 0, 2, 1, 3, 3, 3, 1, 2, 1, 1, 3, 1, 3,
         1, 3, 3, 3, 2, 2, 1, 0, 1, 3, 3, 2, 2, 1, 2, 1, 2, 2, 1, 0, 2, 1, 2, 3,
         3, 0, 3, 2, 2, 0, 0, 3, 0, 2, 1, 2, 3, 2, 1, 3, 0, 2, 0, 3, 3, 0, 3, 1,
         2, 1, 1, 0, 2, 3, 3, 2, 3, 3, 0, 0, 0, 0, 3, 3, 3, 1, 0, 2, 3, 2, 1, 3],
        [2, 0, 1, 1, 2, 3, 1, 0, 2, 2, 1, 0, 0, 1, 2, 1, 1, 2, 1, 2, 2, 0, 3, 1,
         2, 2, 2, 0, 2, 2, 2, 3, 2, 3, 2, 0, 2, 0, 1, 1, 3, 1, 1, 2, 2, 2, 3, 1,
         0, 2, 3, 2, 2, 2,

In [17]:
a

tensor([[[6.3070e-01, 2.2404e-01, 5.4846e-03, 3.5957e-01, 8.9594e-01,
          5.5020e-01, 9.3995e-01, 5.4217e-01, 6.6849e-02, 4.6496e-01,
          8.8257e-01, 4.3787e-01, 4.2099e-01, 4.3236e-01, 6.0123e-01,
          6.6666e-01, 4.9848e-02, 7.3408e-01, 4.3768e-02, 3.3860e-01,
          2.5364e-01, 6.0454e-01, 9.2282e-01, 2.1235e-01, 3.3379e-01,
          2.7169e-01, 5.5289e-03, 8.3089e-01, 7.3584e-01, 2.8690e-01,
          9.6513e-01, 8.0132e-02, 3.9400e-01, 1.7237e-01, 4.2546e-01,
          9.2700e-01, 2.6023e-01, 6.6298e-01, 9.7798e-01, 5.6739e-01,
          3.9487e-01, 1.4848e-01, 5.2263e-01, 1.6742e-01, 4.4138e-01,
          9.9504e-01, 8.0433e-01, 7.3362e-01, 5.1287e-01, 8.3422e-01,
          6.4682e-01, 7.4494e-01, 1.0466e-01, 9.9818e-01, 6.5719e-01,
          9.5469e-01, 8.3655e-01, 8.6032e-02, 5.0289e-01, 3.5904e-01,
          5.3258e-01, 7.8781e-01, 8.6027e-01, 3.1560e-01, 3.0962e-01,
          8.5857e-02, 6.9165e-01, 2.2384e-01, 8.0850e-01, 3.5640e-01,
          8.3193e-01